<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Loading-bulk-datasets" data-toc-modified-id="Loading-bulk-datasets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading bulk datasets</a></span><ul class="toc-item"><li><span><a href="#Regions-in-the-dataset" data-toc-modified-id="Regions-in-the-dataset-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Regions in the dataset</a></span></li><li><span><a href="#Co2-dataset" data-toc-modified-id="Co2-dataset-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Co2 dataset</a></span></li><li><span><a href="#Elec-dataset" data-toc-modified-id="Elec-dataset-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Elec dataset</a></span></li></ul></li><li><span><a href="#Computing-MEFs" data-toc-modified-id="Computing-MEFs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Computing MEFs</a></span><ul class="toc-item"><li><span><a href="#Examples-with-the-MISO-BA" data-toc-modified-id="Examples-with-the-MISO-BA-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Examples with the MISO BA</a></span></li><li><span><a href="#Examples-with-the-CAISO-BA" data-toc-modified-id="Examples-with-the-CAISO-BA-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Examples with the CAISO BA</a></span></li></ul></li><li><span><a href="#Comparison-of-average-MEFs-for-every-BA" data-toc-modified-id="Comparison-of-average-MEFs-for-every-BA-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comparison of average MEFs for every BA</a></span></li><li><span><a href="#A-draft-of-an-hourly-regression-scheme" data-toc-modified-id="A-draft-of-an-hourly-regression-scheme-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>A draft of an hourly regression scheme</a></span><ul class="toc-item"><li><span><a href="#Example-with-MISO" data-toc-modified-id="Example-with-MISO-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Example with MISO</a></span></li><li><span><a href="#Example-with-CISO" data-toc-modified-id="Example-with-CISO-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Example with CISO</a></span></li></ul></li></ul></div>

# Description

In this notebook, I have implemented a rapid prototype for estimating marginal emissions from raw data. The data was downloaded from the following link: web.stanford.edu/~jdechale/emissions_app/. 

Datasets are included within the .zip archive sent alongside the document. 

**A quick note on the structure of the datasets:**

The columns of both datasets have specific names. They are usually a combination of 
- Balacing Authorities ID (e.g. CISO) 
- A tag explaining what we are looking at:
    - D: Demand
    - NG: Net Generation
    - TI: Total Interchange - (positive if exports)
    - ID: Interchange with directly connected balancing authorities - (positive
        if exports)
- For electricity, the type of generation. 

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

from utils import plot_mef, get_BAs, compute_mef

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# replace with your own folder containing the documents

DATA_PATH = '/Users/lucasfuentes/Documents/Ideas/EnergyNetworks/Data'

# Loading bulk datasets

In [ ]:
fnm_co2 = os.path.join(DATA_PATH, 'EBA_co2.csv')
fnm_elec = os.path.join(DATA_PATH, 'EBA_elec.csv')

In [ ]:
df_co2 = pd.read_csv(fnm_co2, index_col=0, parse_dates=True)

df_elec = pd.read_csv(fnm_elec, index_col=0, parse_dates=True)

## Regions in the dataset

In [ ]:
BAs = get_BAs(df_co2)

In [ ]:
print("These are the names of the Balancing Authorities present in the dataset:")
print(BAs)
print(f"\nThere are {len(BAs)} BAs.")

## Co2 dataset

As an example, the first column above gives the CO2-emissions exchanged between AEC and MISO. 

In [ ]:
df_co2.head()

## Elec dataset

The format of the column name is 
- **EBA.ba_name-ALL.[D,NG,TI].H** for the [demand, net_generation, total_interchange] hourly for BA ba_name
- **EBA.ba_name-other_ba_name.ID.H** for the interchange between ba_name and other_ba_name
- **EBA.ba_name-ALL.NG.SOURCE.H** where **SOURCE** is going to be water, wind, coal, ... for the total generation of a given source

In [ ]:
df_elec.head()

For instance, for **MISO**, we see that it exchanges with a bunch of BAs. We also know that it produces nuclear, oil, sun, hydro, wind, and "others"

In [ ]:
for c in df_elec.columns:
    if 'MISO-' in c:
        print(c)

# Computing MEFs

In the simplest models, MEFs $\alpha$ can be computed as
$$
\Delta E \propto \alpha \Delta X,
$$
where: 
- $E$ are the total emissions
- $X$ is an appropriate regressor (total generation, fossil fuel generation, demand...). 

I implemented a simple linear regression as a first prototype. 

## Examples with the MISO BA

Below, we compute the MEF for the MISO BA for different regressors. A discussion of these different figures is provided in the main text. 

In [ ]:
ba = 'MISO'

(ba_, ba_co2), mef, r2 = plot_mef(ba, df_elec, df_co2, which='generation')

In [ ]:
ba = 'MISO'

_ = plot_mef(ba, df_elec, df_co2, which='net_generation')

In [ ]:
ba = 'MISO'

_ = plot_mef(ba, df_elec, df_co2, which='demand')

In [ ]:
ba = 'MISO'

_ = plot_mef(ba, df_elec, df_co2, which='net_demand')

## Examples with the CAISO BA

Below, we compute the MEF for the MISO BA for different regressors. A discussion of these different figures is provided in the main text. 

In [ ]:
ba = 'CISO'

_ = plot_mef(ba, df_elec, df_co2, which='generation')

In [ ]:
ba = 'CISO'

_ = plot_mef(ba, df_elec, df_co2, which='net_generation')

In [ ]:
ba = 'CISO'

_ = plot_mef(ba, df_elec, df_co2, which='demand')

In [ ]:
ba = 'CISO'

_ = plot_mef(ba, df_elec, df_co2, which='net_demand')

# Comparison of average MEFs for every BA

We can apply the above methodology across all BAs, to obtain an overview of the distribution of MEFs. 

In [ ]:
from utils import get_mef_distribution

In [ ]:
which="generation"

In [ ]:
BAs, mefs, r2s = get_mef_distribution(df_elec, df_co2, which=which)

In [ ]:
plt.figure(figsize=(15, 20))
plt.barh(BAs, mefs)
plt.xlabel("MEF (kg/MWh)")
plt.ylabel("BA")
plt.savefig(os.path.join("figs", f"mefs_ALL_{which}.pdf"))

In [ ]:
plt.figure(figsize=(15, 20))
plt.barh(BAs, r2s)
plt.xlabel("R2")
plt.ylabel("BA")
plt.savefig(os.path.join("figs", f"R2_ALL_{which}.pdf"))

# A draft of an hourly regression scheme

In [ ]:
from utils import compute_hourly_mef

## Example with MISO

In [ ]:
ba = 'MISO'

In [ ]:
mefs_hr_G, r2s_hr_G = compute_hourly_mef(ba, df_elec, df_co2, which='generation')

mefs_hr_ND, r2s_hr_ND = compute_hourly_mef(ba, df_elec, df_co2, which='net_demand')

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(mefs_hr_G, 'g-')
ax2.plot(r2s_hr_G, 'b-')
ax1.plot(mefs_hr_ND, 'g--')
ax2.plot(r2s_hr_ND, 'b--')

ax1.set_xlabel('Hour of day')
ax1.set_ylabel('MEF [kg/MWh]', color='g')
ax2.set_ylabel('R2 [-]', color='b')
ax1.set_ylim([0, 1000])
ax2.set_ylim([0, 1])

plt.savefig(os.path.join("figs", f"{ba}_hourly.pdf"));

## Example with CISO

In [ ]:
ba = 'CISO'

In [ ]:
mefs_hr_G, r2s_hr_G = compute_hourly_mef(ba, df_elec, df_co2, which='generation')

mefs_hr_ND, r2s_hr_ND = compute_hourly_mef(ba, df_elec, df_co2, which='net_demand')


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(mefs_hr_G, 'g-')
ax2.plot(r2s_hr_G, 'b-')
ax1.plot(mefs_hr_ND, 'g--')
ax2.plot(r2s_hr_ND, 'b--')

ax1.set_xlabel('Hour of day')
ax1.set_ylabel('MEF [kg/MWh]', color='g')
ax2.set_ylabel('R2 [-]', color='b')
ax1.set_ylim([0, 1000])
ax2.set_ylim([0, 1])

plt.savefig(os.path.join("figs", f"{ba}_hourly.pdf"));